# Projeto 1 - Ciência dos Dados

Nome: Gabriela Kimi Katsurayama

Nome: João Pedro Rodrigues dos Santos

Nome: Luiza Ferreira Ehrenberger

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import nltk
import string
from nltk.corpus import stopwords
#nltk.download('stopwords')
pd.set_option("display.max_colwidth", None)

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\jpthe\Ciencia de dados\CD21-2\P1 git\Cdados1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Iphone 12.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,eu do nada me pego rindo do professor falando de burguesia e tirando um iphone 12 pro max do bolso kkkkk,0
1,"trocarias alguém da escola,por um iphone 12?\n-waweeee eliane.😭",0
2,"sobre o iphone 6s “não ter mais whatsapp”. é td mentira, a maioria ta no ios 12 pra cima, então deixem de alarme ate pq so vai sair se vc tiver no ios 9",3
3,ios 15 ja meu iphone n guenta nem o 12 direito https://t.co/mcshdy359r,4
4,nosso que pechincha 👀🙄uau 🙃\nhttps://t.co/jyhyn71jch,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,"queria lançar o iphone 12 mas é mto dinheiro numa coisa só, o certo é pegar o 11 que ai já da pra pegar agr",4.0
1,ganhei um iphone 12 pro max e o apple watch de presente 😍😍😍😍😍😍😍😍😍😍😍😍,0.0
2,@forsjin iphone? meu namorado pagou 1k no 12 fiquei em choque vius,1.0
3,tem uma criança que tá do meu lado que tem um iphone x e a mãe dele tem um 12 pro max. tô indignado com meu j7 todo quebrado,1.0
4,pensando que é tão besteira pra mim trocar meu iphone 11 pro 12 no natal,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Nós procuramos encontrar um produto de muita relevância no cenário mundial. Depois de muito debate, concluímos que o Iphone 12 era o ideal, uma vez que ele tem muita exposição e estamos na época de anúncio da nova geração.
Colocamos 0 para tweets muito irrelevantes, 1 para tweets irrelevantes, 2 para posts neutros, 3 para posts relevantes e 4 para tweets extremamente relevantes. Tem cerca de 1250 na base total e uma boa porcentagem de tweets neutros/relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
from typing import Awaitable
#Aqui nós estamos definindo as funções. Essa célula contém as funções responsáveis pela limpeza do texto.

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def limpa_txt(txt):
  carac = ['\n', ':', '"', '`', '!', '?', ';','(',')', '[', ']', '%', '&', ',', '.', '@', '/']
  for i in txt:
    if i in carac:
        testador = txt.find(i)
        txt = txt[:testador] + txt[testador+1:]
  vn = arruma_esp(txt)
  return vn

def arruma_esp(txt):
  i = 0
  frase = ''
  aa = 0
  while i < len(txt)+1:
    if i >= 1:
        if i >= len(txt):
            frase += txt[i-1]
            return frase
        else:     
            if txt[i] == ' ' and txt[i-1] == ' ':
                aa += 1
            else:     
                frase += txt[i-1]
    i += 1

def text_preproc(x):
  para = stopwords.words("portuguese")
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in para])
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

def corta_frase(frase):
    tt = frase.lower()
    tt10 = cleanup(tt)
    xa = re.sub(r'\w*\d+\w*', '', tt10)
    xb = re.sub("@\S+", " ", xa)
    xc = re.sub("https*\S+", " ", xb)
    xd = limpa_txt(xc)
    return xd

In [7]:
#Nessa célula tem as funções relacionadas ao processo de obtenção da classificação do tweet.
#A função Classifica faz os cálculos das probabilidades (basicamente o código da aula 7 - Classificador Camões/Lusíadas)

def classifica(frase_cortada):
    resps = []
    i = 0
    probFraseDado0 = 1
    probFraseDado1 = 1
    probFraseDado2 = 1
    probFraseDado3 = 1
    probFraseDado4 = 1

    while i < len(frase_cortada):
        
        
        P_palavra0 = laplace0(frase_cortada[i])
        P_palavra1 = laplace1(frase_cortada[i])
        P_palavra2 = laplace2(frase_cortada[i])
        P_palavra3 = laplace3(frase_cortada[i])
        P_palavra4 = laplace4(frase_cortada[i])
        
        probFraseDado0 *= P_palavra0
        probFraseDado1 *= P_palavra1
        probFraseDado2 *= P_palavra2
        probFraseDado3 *= P_palavra3
        probFraseDado4 *= P_palavra4
        i += 1
    resp0 = probFraseDado0*prob0
    resp1 = probFraseDado1*prob1
    resp2 = probFraseDado2*prob2
    resp3 = probFraseDado3*prob3
    resp4 = probFraseDado4*prob4
        

    resps.append(resp0)
    resps.append(resp1)
    resps.append(resp2)
    resps.append(resp3)
    resps.append(resp4)

    if max(resps) == resp2:
        return 2
    if max(resps) == resp0:
        return 0
    if max(resps) == resp1:
        return 1
    if max(resps) == resp3:
        return 3
    if max(resps) == resp4:
        return 4

#A função Accuracy calcula a precisão do classificador. Para tanto, ela pega os valores retornados da função Classifica e vê se são os mesmos presentes no dataframe.

def accuracy():
    certo = 0
    numer = []
    for index, row in test.iterrows():
        r = classifica(corta_frase(row['Teste']))
        if r == row['Classificação']:
            certo += 1
    numer.append(r)
    return certo/len(test)

#Aqui estão os cálculos da suavização de Laplace. Criamos uma função para cada classificação.
def laplace0(x):
    if x not in tabela_tts_mir:
        qtd = 1
    else:
        qtd = 1 + tabela_tts_mir[x]
        
    deno = len(serie_tweets0) + len(set(serie_tweets_geral))
    return qtd/deno

def laplace1(x):
    if x not in tabela_tts_ir:
        qtd = 1
    else:
        qtd = 1 + tabela_tts_ir[x]
        
    deno = len(serie_tweets1) + len(set(serie_tweets_geral))
    return qtd/deno

def laplace2(x):
    if x not in tabela_tts_n:
        qtd = 1
    else:
        qtd = 1 + tabela_tts_n[x]
        
    deno = len(serie_tweets2) + len(set(serie_tweets_geral))
    return qtd/deno

def laplace3(x):
    if x not in tabela_tts_r:
        qtd = 1
    else:
        qtd = 1 + tabela_tts_r[x]
        
    deno = len(serie_tweets3) + len(set(serie_tweets_geral))
    return qtd/deno

def laplace4(x):
    if x not in tabela_tts_mr:
        qtd = 1
    else:
        qtd = 1 + tabela_tts_mr[x]
        
    deno = len(serie_tweets5) + len(set(serie_tweets_geral))
    return qtd/deno

In [8]:
#Aqui é o começo do caminho de classificação. 
#O primeiro passo é criar dataframes só com os tweets de cada categoria
#Fazemos isso usando a função loc do pandas

filtra_linhas_0 = train['Classificação'] == 0
mir = train.loc[filtra_linhas_0, :]

filtra_linhas_1 = train['Classificação'] == 1
ir = train.loc[filtra_linhas_1, :]

filtra_linhas_2 = train['Classificação'] == 2
n = train.loc[filtra_linhas_2, :]

filtra_linhas_3 = train['Classificação'] == 3
r = train.loc[filtra_linhas_3, :]

filtra_linhas_4 = train['Classificação'] == 4
mr = train.loc[filtra_linhas_4, :]


#/----------------------------------------------------/

#Agora que temos dataframes com tweets de mesma classificação, vamos começar a limpar os textos
#O primeiro passo é transformar todas as linhas da coluna "Treinamento" em uma string MUITO grande

m_i_r = mir['Treinamento'].to_string().lower()

#Agora nós passamos a string na função que limpa URLs, menções e outros marcadores que podem prejudicar o classificador
#Como faz parte da rubrica, nós deixamos os emojis dentro da string

m_i_r2 = text_preproc(m_i_r)

#Essa linha agora simplesmente separa a string em uma lista gigante de palavras
m_i_r3 = m_i_r2.split()

#Criamos uma Series para usar o value_counts
serie_tweets0 = pd.Series(m_i_r3)

#Por fim, criamos uma tabela de tweets relativa (mostra a porcentagem da palavra) e uma geral (mostra quantas vezes apareceu)
tabela_tts_mir_relativa = serie_tweets0.value_counts(True)
tabela_tts_mir = serie_tweets0.value_counts()

#Repetimos esse processo para todas as classes

ir_1 = ir['Treinamento'].to_string().lower()
ir2 = text_preproc(ir_1)
ir3 = ir2.split()
serie_tweets1 = pd.Series(ir3)
tabela_tts_ir_relativa = serie_tweets1.value_counts(True)
tabela_tts_ir = serie_tweets1.value_counts()

n_1 = n['Treinamento'].to_string().lower()
n2 = text_preproc(n_1)
n3 = n2.split()
serie_tweets2 = pd.Series(n3)
tabela_tts_n_relativa = serie_tweets2.value_counts(True)
tabela_tts_n = serie_tweets2.value_counts()

r_1 = r['Treinamento'].to_string().lower()
r_2 = text_preproc(r_1)
r_3 = r_2.split()
serie_tweets3 = pd.Series(r_3)
tabela_tts_r_relativa = serie_tweets3.value_counts(True)
tabela_tts_r = serie_tweets3.value_counts()

mr_1 = mr['Treinamento'].to_string().lower()
mr2 = text_preproc(mr_1)
mr3 = mr2.split()
serie_tweets5 = pd.Series(mr3)
tabela_tts_mr_relativa = serie_tweets5.value_counts(True)
tabela_tts_mr = serie_tweets5.value_counts()

geral = train['Treinamento'].to_string().lower()
geral2 = text_preproc(geral)
geral3 = geral2.split()
serie_tweets_geral = pd.Series(geral3)
tabela_tts_geral_relativa = serie_tweets_geral.value_counts(True)
tabela_tts_geral = serie_tweets_geral.value_counts()

In [9]:
#Essa célula cria a variável universal. Ela tem o valor da quantidade de todas as palavras presentes nos tweets.
todos = [m_i_r2, ir2, n2, r_2, mr2]
i = 0
tabela_geral = ''
while i < len(todos):
    tabela_geral += todos[i]
    i += 1

i = 0
universal = 0
while i < len(todos):
    universal += len(todos[i])
    i += 1

todas_palavras = tabela_geral.split()
serie_geral = pd.Series(todas_palavras)

tabela_geral_relativa = serie_geral.value_counts(True)


In [10]:
#Aqui nós pegamos um tweet aleatório para testar
tt = test['Teste'].sample().to_string().lower()
print(tt)

tt10 = cleanup(tt)
xa = re.sub(r'\w*\d+\w*', '', tt10)
xb = re.sub("@\S+", " ", xa)
xc = re.sub("https*\S+", " ", xb)
xd = limpa_txt(xc)
print(xd)

149    na dúvida de qual comprar  ... iphone 12 ou samsung note 20 🤔🤔🤔
 na dúvida de qual comprar iphone ou samsung note 🤔🤔🤔


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
frase_cortada = xd.split()


print(frase_cortada)

['na', 'dúvida', 'de', 'qual', 'comprar', 'iphone', 'ou', 'samsung', 'note', '🤔🤔🤔']


In [12]:
prob0 = len(m_i_r2)/universal
prob1 = len(ir2)/universal
prob2 = len(n2)/universal
prob3 = len(r_2)/universal
prob4 = len(mr2)/universal
print(prob0,prob1,prob2,prob3,prob4)

0.3358807199226426 0.23445834243114258 0.14279921394928102 0.11711219938238872 0.16974952431454507


In [13]:
print(classifica(frase_cortada))

4


In [14]:
#Aqui nós passamos a testar com todos os elementos da tabela test. Após isso começamos a cruzar os dados.
certo = 0
numer = []
for index, row in test.iterrows():
    r = classifica(text_preproc(row['Teste']).split())
    if r == row['Classificação']:
        certo += 1
    numer.append(r)
print(certo/len(test))


0.4498714652956298


___
### Concluindo

Esse classificador Naive Bayes tem acurácia de aproximadamente 45%. Para ter um melhor desempenho acreditamos que uma base de dados maior é crucial, uma vez que a atual tem apenas 1323 tweets.
Como está ainda em desenvolvimento, se usássemos esse classificador para gerar novos testes, os resultados seriam muito pouco confiáveis, causando mais trabalho humano para conferir os erros do programa.

Classificadores assim podem ser usados em diversos setores. Eles podem estar presentes desde campanhas eleitorais, um grande exemplo é a eleição americana de 2008, em que a equipe de Obama utilizou inteligência artificial para calcular as áreas mais prováveis dele conseguir mais votos. Além disso, esses códigos podem servir para fazer sugestões de vendas para um cliente com base no seu histórico. 

Como o código não é perfeito e não consegue entender algumas nuances da língua, o uso de um classificador para gerar mais amostras de treinamento seria um grande erro, uma vez que o programa criaria uma base de dados enviesada. Isso faria com que o classificador ficasse totalmente perdido, visto que ele começaria a classificar tweets de maneira errada e isso bagunçaria sua base de dados.

In [15]:
#pd.crosstab
cross = test
cross['Classificador'] = numer
pd.crosstab(cross.Classificador,cross.Classificação)

Classificação,0.0,1.0,2.0,3.0,4.0
Classificador,,,,,
0,134,24,13,9,4
1,61,31,13,9,4
2,14,4,3,2,3
3,11,4,2,2,0
4,18,5,2,11,5


In [16]:
#Aqui tem um dataframe que mostra a classificação manual x classificação do programa
cross

,Teste,Classificação,Classificador
0,"queria lançar o iphone 12 mas é mto dinheiro numa coisa só, o certo é pegar o 11 que ai já da pra pegar agr",4.0,1
1,ganhei um iphone 12 pro max e o apple watch de presente 😍😍😍😍😍😍😍😍😍😍😍😍,0.0,0
2,@forsjin iphone? meu namorado pagou 1k no 12 fiquei em choque vius,1.0,1
3,tem uma criança que tá do meu lado que tem um iphone x e a mãe dele tem um 12 pro max. tô indignado com meu j7 todo quebrado,1.0,0
4,pensando que é tão besteira pra mim trocar meu iphone 11 pro 12 no natal,1.0,0
...,...,...,...
384,devo vender toda minha coleção pra comprar um iphone 12 pro max 512gb?,2.0,1
385,eu preciso de um iphone 12 😪,0.0,1
386,tem um cara falando q vai me dar um iphone 12 pro max 😢😢😢😢😢😢😢 isso tem q ser vdd,2.0,1
387,@kiki_djavando era só um iphone 12,0.0,0


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [17]:
#sk.learn


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Cleaning Text Data with Python](https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76) **Para formatação do texto**